In [ ]:
from pathlib import Path

import evaluate
from datasets import concatenate_datasets
from transformers import IntervalStrategy

from utils.dataset_utils import get_datasets_for_leave_one_out_benchmark, \
    take_even_class_distribution_sample, \
    transform_for_inference
from utils.model_utils import POLITICAL_LEANING_WITH_CENTER_LABEL_MAPPING, finetune_custom_models

In [ ]:
TRAIN_DATASET_SAMPLE_SIZE = 1_000
EVAL_DATASET_SAMPLE_SIZE = 100

train_datasets_separate = []
eval_datasets = []
for dataset in get_datasets_for_leave_one_out_benchmark():
    label_mapping = POLITICAL_LEANING_WITH_CENTER_LABEL_MAPPING

    eval_dataset = take_even_class_distribution_sample(dataset, EVAL_DATASET_SAMPLE_SIZE)
    eval_dataset = transform_for_inference(eval_dataset, label_mapping)

    # Remove the eval sample from the source dataframe.
    dataset.dataframe = dataset.dataframe.loc[~dataset.dataframe.index.isin(eval_dataset.dataframe.index)]

    train_dataset = take_even_class_distribution_sample(dataset, TRAIN_DATASET_SAMPLE_SIZE)
    train_dataset = transform_for_inference(train_dataset, label_mapping)

    train_datasets_separate.append(train_dataset.to_huggingface())
    eval_datasets.append(eval_dataset.to_huggingface())

train_datasets = [
    concatenate_datasets(
        list(filter(
            lambda dataset: dataset.info.dataset_name != left_out_dataset.info.dataset_name, train_datasets_separate
        )),
        info=left_out_dataset.info
    ) for left_out_dataset in train_datasets_separate
]

In [ ]:
TRAINING_SEED = 37
DATA_SEED = 37
METRIC = evaluate.load("accuracy")
EVAL_STRATEGY = IntervalStrategy.EPOCH

finetune_custom_models(
    Path("dataset_benchmark", "leave_one_out"),
    train_datasets,
    eval_datasets,
    METRIC,
    EVAL_STRATEGY,
    TRAINING_SEED,
    DATA_SEED
)